In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import os

from scipy import stats
import joblib

In [2]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

# Crossvalidation support cells


In [3]:
from sklearn.metrics import mean_absolute_error,make_scorer

def mae_exp(y_true_log, y_pred_log):
    y_true = np.exp(y_true_log)
    y_pred = np.exp(y_pred_log)
    return mean_absolute_error(y_true, y_pred)

# Create a scorer from the custom scoring function
mae_exp_scorer = make_scorer(mae_exp, greater_is_better=False)

In [4]:
result = {}

# Load dataset

In [5]:
dataframe = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\毕业论文2024\Notebook\Data\without_fake\original.csv')

# cross_features = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\毕业论文2024\Notebook\Data\cross_features.csv')
# cf_list = ['total_brain_vol','gray_to_white_ratio','total_surface_area','volume_surface_ratio','thickness_volume_ratio','volume_weighted_cur_mean']
# dataframe = pd.concat([dataframe,cross_features[cf_list]],axis = 1)

,aseg_Left-Cerebellum-White-Matter,aseg_Left-Cerebellum-Cortex,aseg_Left-Thalamus,aseg_Left-Caudate,aseg_Left-Putamen,aseg_Left-Pallidum,aseg_Brain-Stem,aseg_Left-Hippocampus,aseg_Left-Amygdala,aseg_Left-Accumbens-area,...,wmparc_wm-rh-superiorfrontal,wmparc_wm-rh-superiorparietal,wmparc_wm-rh-superiortemporal,wmparc_wm-rh-supramarginal,wmparc_wm-rh-frontalpole,wmparc_wm-rh-temporalpole,wmparc_wm-rh-transversetemporal,wmparc_wm-rh-insula,wmparc_Left-UnsegmentedWhiteMatter,wmparc_Right-UnsegmentedWhiteMatter
0,13706.1,50274.2,7559.5,3314.4,4922.1,1671.2,17953.4,4179.7,1491.3,540.5,...,18565.4,11848.3,8129.0,8736.6,381.7,723.3,636.2,10862.9,34016.5,32253.2
1,11981.9,51047.5,8563.6,3525.5,5072.9,1633.4,17662.2,4247.6,1446.9,477.8,...,15577.1,10289.2,6145.2,8386.2,307.4,715.4,496.3,8409.0,30334.0,33284.7
2,21259.1,62010.0,9594.8,3376.6,5125.7,2386.0,22903.6,4604.2,1595.9,337.6,...,22376.1,11366.6,8323.6,9290.7,484.2,734.7,613.3,10585.0,34007.3,32710.9
3,21745.4,54823.7,8708.1,4503.2,6281.9,2259.7,22801.7,4925.7,1868.9,518.3,...,18305.7,12225.8,9422.6,9690.4,452.8,844.0,654.0,11146.3,31778.2,30407.8
4,17233.3,57743.1,9288.5,3706.9,5902.9,2511.3,21597.8,4795.1,1779.5,477.0,...,17272.5,12557.2,7031.8,8993.7,395.9,1033.6,600.9,10615.9,34509.2,32888.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,13786.4,54537.3,7902.8,3075.6,4129.8,1991.4,20980.4,4229.0,1639.5,338.8,...,16386.7,14765.0,7485.8,9449.4,345.1,930.7,861.4,9895.0,35185.6,35635.4
1596,14876.8,55987.1,8009.5,3343.2,5091.7,1974.9,21835.5,3630.9,1569.1,475.1,...,17264.9,12851.4,6271.0,8483.7,347.3,642.3,779.0,9703.9,35090.9,34014.9
1597,15533.6,52459.5,8732.0,3410.0,5185.2,2125.4,21134.2,4051.4,1352.5,465.9,...,18231.9,13271.8,6718.9,6707.7,406.5,857.8,676.4,9093.7,29642.8,30435.8
1598,22349.1,56921.2,10410.8,4130.0,5464.3,2476.4,22135.7,4073.4,1721.0,557.7,...,21797.0,12247.4,8580.3,9761.5,470.4,763.9,666.0,9403.8,34844.6,35012.3


# Variance


In [7]:
variance = dataframe.var(axis = 0)
variance.describe()

count    4.100000e+02
mean     1.454364e+06
std      7.642431e+06
min      7.586233e-05
25%      3.150134e-02
50%      8.404058e+04
75%      4.571961e+05
max      1.024390e+08
dtype: float64

In [8]:
dataframe = dataframe.drop(variance[variance <0.001].index,axis = 1)
dataframe.shape

(1600, 349)

In [9]:
dataframe

,aseg_Left-Cerebellum-White-Matter,aseg_Left-Cerebellum-Cortex,aseg_Left-Thalamus,aseg_Left-Caudate,aseg_Left-Putamen,aseg_Left-Pallidum,aseg_Brain-Stem,aseg_Left-Hippocampus,aseg_Left-Amygdala,aseg_Left-Accumbens-area,...,wmparc_wm-rh-superiorfrontal,wmparc_wm-rh-superiorparietal,wmparc_wm-rh-superiortemporal,wmparc_wm-rh-supramarginal,wmparc_wm-rh-frontalpole,wmparc_wm-rh-temporalpole,wmparc_wm-rh-transversetemporal,wmparc_wm-rh-insula,wmparc_Left-UnsegmentedWhiteMatter,wmparc_Right-UnsegmentedWhiteMatter
0,13706.1,50274.2,7559.5,3314.4,4922.1,1671.2,17953.4,4179.7,1491.3,540.5,...,18565.4,11848.3,8129.0,8736.6,381.7,723.3,636.2,10862.9,34016.5,32253.2
1,11981.9,51047.5,8563.6,3525.5,5072.9,1633.4,17662.2,4247.6,1446.9,477.8,...,15577.1,10289.2,6145.2,8386.2,307.4,715.4,496.3,8409.0,30334.0,33284.7
2,21259.1,62010.0,9594.8,3376.6,5125.7,2386.0,22903.6,4604.2,1595.9,337.6,...,22376.1,11366.6,8323.6,9290.7,484.2,734.7,613.3,10585.0,34007.3,32710.9
3,21745.4,54823.7,8708.1,4503.2,6281.9,2259.7,22801.7,4925.7,1868.9,518.3,...,18305.7,12225.8,9422.6,9690.4,452.8,844.0,654.0,11146.3,31778.2,30407.8
4,17233.3,57743.1,9288.5,3706.9,5902.9,2511.3,21597.8,4795.1,1779.5,477.0,...,17272.5,12557.2,7031.8,8993.7,395.9,1033.6,600.9,10615.9,34509.2,32888.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,13786.4,54537.3,7902.8,3075.6,4129.8,1991.4,20980.4,4229.0,1639.5,338.8,...,16386.7,14765.0,7485.8,9449.4,345.1,930.7,861.4,9895.0,35185.6,35635.4
1596,14876.8,55987.1,8009.5,3343.2,5091.7,1974.9,21835.5,3630.9,1569.1,475.1,...,17264.9,12851.4,6271.0,8483.7,347.3,642.3,779.0,9703.9,35090.9,34014.9
1597,15533.6,52459.5,8732.0,3410.0,5185.2,2125.4,21134.2,4051.4,1352.5,465.9,...,18231.9,13271.8,6718.9,6707.7,406.5,857.8,676.4,9093.7,29642.8,30435.8
1598,22349.1,56921.2,10410.8,4130.0,5464.3,2476.4,22135.7,4073.4,1721.0,557.7,...,21797.0,12247.4,8580.3,9761.5,470.4,763.9,666.0,9403.8,34844.6,35012.3


Cross features


# train test split

In [10]:
# Train test split
from sklearn.model_selection import train_test_split


#log transform
y = (dataframe['age'])
X = dataframe.drop('age',axis = 1)

print(dataframe.shape)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# data standardization
from sklearn.preprocessing import StandardScaler
x_train_scaled = StandardScaler().fit_transform(x_train)
x_test_scaled = StandardScaler().fit_transform(x_test)


(1600, 349)


# Vanilla Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)

LinearRegression()

In [12]:
joblib.dump(lr,r'C:\Users\User\OneDrive\Desktop\毕业论文2024\Notebook\joblib\lr.joblib')

['C:\\Users\\User\\OneDrive\\Desktop\\毕业论文2024\\Notebook\\joblib\\lr.joblib']

In [13]:
from sklearn.model_selection import cross_val_score
cv_lr = cross_val_score(lr, x_train_scaled, y_train, cv=10, scoring='neg_mean_absolute_error')
score = -np.mean(cv_lr)
result[f'vanilla_original_{dataframe.columns.shape[0]}']=score

In [14]:
print(result)


{'vanilla_original_349': 10.354568876905619}


In [15]:
from sklearn.metrics import mean_absolute_error
y_pred = lr.predict(x_test_scaled)
mae = mean_absolute_error((y_test), (y_pred))
print("vanilla_test:", mae)
result[f'vanilla_test{dataframe.columns.shape[0]}'] = mae

vanilla_test: 9.360871120417972


# Lasso + CV + GridSearch for alpha

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV


lasso = Lasso()
# Define a range of alpha values
alpha_range = np.logspace(-4, -0.5, 20)

# Set up GridSearchCV
param_grid = {'alpha': alpha_range}
grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=10, scoring='neg_mean_absolute_error')

# Fit GridSearchCV
grid_search.fit(x_train_scaled, y_train)

# Best alpha value
print("Best alpha:", grid_search.best_params_['alpha'])

# Evaluate on the test set
best_lasso = grid_search.best_estimator_
cv_bestlasso = cross_val_score(best_lasso,x_train_scaled,y_train,cv = 10,scoring = 'neg_mean_absolute_error')


score = -np.mean(cv_bestlasso)
print(f'score:{score} vs gridscore:{-grid_search.best_score_}')
result[f'lassoGS_original{dataframe.columns.shape[0]}'] = score


Best alpha: 0.13538761800225432
score:9.348039521360493 vs gridscore:9.348039521360493


In [17]:
joblib.dump(best_lasso,r'C:\Users\User\OneDrive\Desktop\毕业论文2024\Notebook\joblib\best_lasso.joblib')

['C:\\Users\\User\\OneDrive\\Desktop\\毕业论文2024\\Notebook\\joblib\\best_lasso.joblib']

In [18]:
print(result)

{'vanilla_original_349': 10.354568876905619, 'vanilla_test349': 9.360871120417972, 'lassoGS_original349': 9.348039521360493}


# Feature Selection using lasso coef


In [19]:
# from sklearn.feature_selection import SelectFromModel
# model = SelectFromModel(best_lasso)
# new_features = model.transform(x_train)
# print()

In [20]:
# # Get coefficients
coef = pd.Series(best_lasso.coef_, index=X.columns)
lasso_features_selection = coef[coef != 0].index.tolist()
print(len(lasso_features_selection))
%store lasso_features_selection

129
Stored 'lasso_features_selection' (list)


# Test Result

In [21]:
y_pred = best_lasso.predict(x_test_scaled)
mae = mean_absolute_error((y_test), (y_pred))
print("lasso_test:", mae)
result[f'lasso_test{dataframe.columns.shape[0]}'] = mae
print(result)


lasso_test: 8.52515176008671
{'vanilla_original_349': 10.354568876905619, 'vanilla_test349': 9.360871120417972, 'lassoGS_original349': 9.348039521360493, 'lasso_test349': 8.52515176008671}


### lasso filter features 

In [22]:

lasso_features_selection.append('age')
dataframe = dataframe[lasso_features_selection]

# Train test split
from sklearn.model_selection import train_test_split


#log transform
y = (dataframe['age'])
X = dataframe.drop('age',axis = 1)

print(dataframe.shape)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# data standardization
from sklearn.preprocessing import StandardScaler
x_train_scaled = StandardScaler().fit_transform(x_train)
x_test_scaled = StandardScaler().fit_transform(x_test)


(1600, 130)


# Elastic net using lasso filtered features

In [23]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet()
# Define a range of alpha values
alpha_range = np.logspace(-4, -0.5, 20)
l1_range = np.logspace(-4 , - 0.5, 20)

# Set up GridSearchCV
param_grid = {'alpha': alpha_range,'l1_ratio' : l1_range}
grid_search_e = GridSearchCV(estimator=enet, param_grid=param_grid, cv=10, scoring='neg_mean_absolute_error')

# Fit GridSearchCV
grid_search_e.fit(x_train_scaled, y_train)

# Best alpha value
print("Best alpha:", grid_search_e.best_params_['alpha'])
print("Best l1_ratio:", grid_search_e.best_params_['l1_ratio'])
# Evaluate on the test set
best_enet = grid_search_e.best_estimator_
cv_bestenet = cross_val_score(best_enet,x_train_scaled,y_train,cv = 10,scoring = 'neg_mean_absolute_error')

score = -np.mean(cv_bestenet)
result[f'enetGS_original{dataframe.columns.shape[0]}'] = score
y_pred = best_enet.predict(x_test_scaled)
mae = mean_absolute_error((y_test), (y_pred))
print("enet_test:", mae)
result[f'enet_test{dataframe.columns.shape[0]}'] = mae
print(result)


Best alpha: 0.0379269019073225
Best l1_ratio: 0.0001
enet_test: 8.567572564309813
{'vanilla_original_349': 10.354568876905619, 'vanilla_test349': 9.360871120417972, 'lassoGS_original349': 9.348039521360493, 'lasso_test349': 8.52515176008671, 'enetGS_original130': 8.918593086800835, 'enet_test130': 8.567572564309813}


In [24]:
joblib.dump(best_enet,r'C:\Users\User\OneDrive\Desktop\毕业论文2024\Notebook\joblib\best_enet.joblib')

['C:\\Users\\User\\OneDrive\\Desktop\\毕业论文2024\\Notebook\\joblib\\best_enet.joblib']

# Ridge Regression - using features extracted by lasso


In [25]:
from sklearn.linear_model import Ridge

ridge = Ridge()
# Define a range of alpha values
alpha_range = np.logspace(-4, -0.5, 20)

# Set up GridSearchCV
param_grid = {'alpha': alpha_range}
grid_search_r = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=10, scoring='neg_mean_absolute_error')

# Fit GridSearchCV
grid_search_r.fit(x_train_scaled, y_train)

# Best alpha value
print("Best alpha:", grid_search_r.best_params_['alpha'])

# Evaluate on the test set
best_ridge = grid_search_r.best_estimator_
cv_bestridge = cross_val_score(best_ridge,x_train_scaled,y_train,cv = 10,scoring = 'neg_mean_absolute_error')

score = -np.mean(cv_bestridge)
result[f'RidgeGS_original{dataframe.columns.shape[0]}'] = score


Best alpha: 0.31622776601683794


In [26]:
joblib.dump(best_ridge,r'C:\Users\User\OneDrive\Desktop\毕业论文2024\Notebook\joblib\lasso+ridge.joblib')

['C:\\Users\\User\\OneDrive\\Desktop\\毕业论文2024\\Notebook\\joblib\\lasso+ridge.joblib']

In [27]:
y_pred = best_ridge.predict(x_test_scaled)
mae = mean_absolute_error((y_test), (y_pred))
print("ridge_test:", mae)
result[f'ridge_test{dataframe.columns.shape[0]}'] = mae
print(result)


ridge_test: 8.743068880336148
{'vanilla_original_349': 10.354568876905619, 'vanilla_test349': 9.360871120417972, 'lassoGS_original349': 9.348039521360493, 'lasso_test349': 8.52515176008671, 'enetGS_original130': 8.918593086800835, 'enet_test130': 8.567572564309813, 'RidgeGS_original130': 8.957469547154306, 'ridge_test130': 8.743068880336148}


Append to result.csv

In [28]:
result = pd.Series(result)
result.to_csv(r'C:\\Users\User\\OneDrive\Desktop\毕业论文2024\\Notebook\\result.csv',mode='a')